In [ ]:
# To install pref_voting, run on command line:
# pip install pref_voting

In [2]:
import json
import pandas as pd
import numpy as np
from pref_voting.profiles_with_ties import *
from pref_voting.voting_methods import *
from pref_voting.iterative_methods import instant_runoff, coombs
from pref_voting.c1_methods import condorcet

import csv
from collections import Counter, defaultdict
from pref_voting.profiles import Profile


In [3]:
# Function to convert rankings (in tuple format) to indices and create dictionaries as keys
def ranking_to_indices(ranking, candidate_to_index):
    ranking_dict = {}
    for rank, candidate in enumerate(ranking, start=1):
        ranking_dict[candidate_to_index.get(candidate, -1)] = rank
    return ranking_dict

def create_profile(file_path):
    election_data = pd.read_csv(file_path)
    column_names_list = election_data.columns.tolist()
    num_ranks=0
    for item in column_names_list:
        if 'rank' in item:
            num_ranks+=1

    # Function to process each voter's rankings, ignoring "overvote"
    def process_rankings(row):
        seen_candidates = set()
        ranking = defaultdict(list)
        rank = 1
        
        for i in range(1, num_ranks+1):
            candidate = row[f'rank{i}']
            if candidate != 'overvote' and candidate != 'skipped' and candidate not in seen_candidates:
                ranking[rank].append(candidate)
                seen_candidates.add(candidate)
                rank += 1
        
        return {c: r for r, cs in ranking.items() for c in cs}

    # Apply the function and count occurrences of each ranking
    election_data['processed_rankings'] = election_data.apply(process_rankings, axis=1)
    ranking_counts = Counter(election_data['processed_rankings'].apply(tuple))

    # Create candidate list and set "Write-in" index last
    candidates = [c for c in set().union(*[r for r in ranking_counts])]

    # Convert rankings to indices format
    candidate_to_index = {candidate: index for index, candidate in enumerate(candidates)}

    # Display the candidates and their indices for reference
    candidates_with_indices = {index: candidate for candidate, index in candidate_to_index.items()}
    print(candidates_with_indices)

    # Creating rankings with indices, ignoring ballots with "overvote"
    rankings = [ranking_to_indices(ranking, candidate_to_index) for ranking, count in ranking_counts.items()]
    rcounts = [count for ranking, count in ranking_counts.items()]

    # Create ProfileWithTies object
    profile = ProfileWithTies(rankings, rcounts)
    # profile.display()

    # Treat unranked candidates as tied for last place below ranked candidates for the purposes of the margin graph
    # profile.use_extended_strict_preference()

    # Display the margin graph
    # profile.display_margin_graph()

    return profile

profile = create_profile('/Users/belle/Desktop/build/rcv_proposal/scotland/processed_data/aberdeen2017/AiryhallBroomhillGarthdeeWard_aberdeen17-11.csv')

{0: 'Ian YUILL (LD)', 1: 'Angela TAYLOR (Lab)', 2: 'Douglas LUMSDEN (Con)', 3: 'Gordon Scott TOWNSON (SNP)'}


In [78]:
def run_voting_methods(profile):
    data = {}
    ## PLURALITY ##
    # The Plurality score of a candidate c is the number of voters that rank c in first place. 
    # The Plurality winners are the candidates with the largest Plurality score in the profile.
    plurality.display(profile)
    data['plurality'] = plurality(profile)[0]

    ## Plurality With Runoff PUT ##
    # If there is a majority winner then that candidate is the Plurality with Runoff winner. 
    # Otherwise hold a runoff between the top two candidates: the candidate with the most first place votes 
    # and the candidate with the 2nd most first place votes (or perhaps tied for the most first place votes). 
    # In the case of multiple candidates tied for the most or 2nd most first place votes, use parallel-universe 
    # tiebreaking: a candidate is a Plurality with Runoff winner if it is a winner in some runoff as described. 
    # If the candidates are all tied for the most first place votes, then all candidates are winners.
    plurality_with_runoff_put.display(profile)
    data['plurality_with_runoff_put'] = plurality_with_runoff_put(profile)[0]

    ## Instant Runoff for Truncated Linear Orders ##
    # Iteratively remove the candidates with the fewest number of first place votes, until there is a candidate 
    # with more than the threshold number of first-place votes. If a threshold is not set, then it is strictly 
    # more than half of the non-empty ballots.
    instant_runoff_for_truncated_linear_orders.display(profile)
    data['instant_runoff_for_truncated_linear_orders'] = instant_runoff_for_truncated_linear_orders(profile)[0]

    ## Bottom-Two-Runoff Instant Runoff PUT ##
    # Find the two candidates with the lowest two plurality scores, remove the one 
    # who loses head-to-head to the other, and repeat until a single candidate remains. 
    # Parallel-universe tiebreaking is used to break ties for lowest or second lowest plurality scores.
    bottom_two_runoff_instant_runoff_put.display(profile)
    data['bottom_two_runoff_instant_runoff_put'] = bottom_two_runoff_instant_runoff_put(profile)[0]

    ## Instant Runoff PUT ##
    # Instant Runoff (instant_runoff()) with parallel universe tie-breaking (PUT), defined recursively: 
    # if there is a candidate with a strict majority of first-place votes, that candidate is the IRV-PUT winner; 
    # otherwise a candidate x is an IRV-PUT winner if there is some candidate y with a minimal number of 
    # first-place votes such that after removing y from the profile, x is an IRV-PUT winner.
    instant_runoff_put.display(profile)
    data['instant_runoff_put'] = instant_runoff_put(profile)[0]

    ## BORDA ##
    # Borda score for truncated linear orders using different ways of defining the Borda score for truncated linear orders.
    borda_for_profile_with_ties.display(profile)
    data['borda_for_profile_with_ties'] = borda_for_profile_with_ties(profile)[0]

    ## To be added: BUCKLIN (currently runs into AttributeError: 'ProfileWithTies' object has no attribute '_rcounts') ##

    ## CONDORCET ##
    # Return the Condorcet winner if one exists, otherwise return all the candidates. A Condorcet winner is a candidate c
    # that is majority preferred to every other candidate c
    condorcet.display(profile)
    data['condorcet'] = condorcet(profile)[0]

    ## CONDORCET IRV ##
    # If a Condorcet winner exists, elect that candidate, otherwise return the instant runoff winners.
    condorcet_irv.display(profile)
    data['condorcet_irv'] = condorcet_irv(profile)[0]

    ## WEAK CONDORCET ##
    # Return all weak Condorcet winner if one exists, otherwise return all the candidates. A weak Condorcet winner is a candidate 
    # c such that no other candidate is majority preferred to c.
    weak_condorcet.display(profile)
    data['weak_condorcet'] = weak_condorcet(profile)[0]

    ## BENHAM ##
    # As long as the profile has no Condorcet winner, eliminate the candidate with the lowest plurality score.
    benham.display(profile)
    data['benham'] = benham(profile)[0]

    ## MINIMAX ##
    # The Minimax winners are the candidates with the smallest maximum pairwise loss.  
    # That is, for each candidate a, find the biggest margin of a candidate 
    # a over b, then elect the candidate(s) with the smallest such loss. Also known as the Simpson-Kramer Rule.
    minimax.display(profile)
    data['minimax'] = minimax(profile)[0]

    ## Ranked Pairs ##
    # Order the edges in the margin graph from largest to smallest and lock them in in that order, 
    # skipping edges that create a cycle. If there are ties in the margins, break the ties using a 
    # tie-breaking rule: a linear ordering over the edges. A candidate is a Ranked Pairs winner if 
    # it wins according to some tie-breaking rule. Also known as Tideman’s Rule.
    ranked_pairs.display(profile)
    data['ranked_pairs'] = ranked_pairs(profile)[0]

    ## TOP CYCLE ##
    # The smallest set of candidates such that every candidate inside the set is majority preferred to every candidate outside the set.
    top_cycle.display(profile)
    data['top_cycle'] = top_cycle(profile)[0]

    ## COPELAND ##
    # The Copeland score for c is the number of candidates that c is majority preferred to minus the number of candidates majority preferred to c. 
    # The Copeland winners are the candidates with the maximum Copeland score in the profile restricted to curr_cands.
    if len(copeland(profile))==1:
        copeland.display(profile)
        data['copeland'] = copeland(profile)[0]

    ## DAUNOU ##
    # Implementation of Daunou’s voting method as described in the paper: https://link.springer.com/article/10.1007/s00355-020-01276-w
    daunou(profile)
    data['daunou'] = daunou(profile)[0]

    return data

data = run_voting_methods(profile)
print(data)

Plurality winner is {2}
PluralityWRunoff PUT winner is {2}
Instant Runoff (Truncated Linear Orders) winner is {2}
Bottom-Two-Runoff Instant Runoff PUT winner is {2}
Instant Runoff PUT winner is {2}
Borda (for Truncated Profiles) winner is {2}
Condorcet winner is {2}
Condorcet IRV winner is {2}
Weak Condorcet winner is {2}
Benham winner is {2}
Minimax winner is {2}
Ranked Pairs winner is {2}
Top Cycle winner is {2}
Copeland winner is {2}
{'plurality': 2, 'plurality_with_runoff_put': 2, 'instant_runoff_for_truncated_linear_orders': 2, 'bottom_two_runoff_instant_runoff_put': 2, 'instant_runoff_put': 2, 'borda_for_profile_with_ties': 2, 'condorcet': 2, 'condorcet_irv': 2, 'weak_condorcet': 2, 'benham': 2, 'minimax': 2, 'ranked_pairs': 2, 'top_cycle': 2, 'copeland': 2, 'daunou': 2}
